# Turbo Stage Example


In [3]:
import numpy as np
from parafoil import TurboRowPassage, TurboStagePassage, TurboMeshParameters
from ezmesh import visualize_mesh
from parafoil import CamberThicknessAirfoil
from paraflow import run_simulation, get_flasher, SimulationOptions, get_frames, display_frame

rotor_airfoil = CamberThicknessAirfoil(
    inlet_angle=np.radians(-60),
    outlet_angle=np.radians(-51),
    upper_thick_prop=[0.015, 0.05, 0.05  , 0.015],
    lower_thick_prop=[0.015, 0.05, 0.05, 0.015],
    leading_prop=0.5,
    trailing_prop=0.5,
    chord_length=0.018,
)

stator_airfoil = CamberThicknessAirfoil(
    inlet_angle=np.radians(26),
    outlet_angle=np.radians(12),
    upper_thick_prop=[0.015, 0.05, 0.05  , 0.015],
    lower_thick_prop=[0.015, 0.05, 0.05, 0.015],
    leading_prop=0.5,
    trailing_prop=0.5,
    chord_length=0.018,
)

axial_turbo_passage = TurboStagePassage(
    inflow_passage=TurboRowPassage(
        airfoil=rotor_airfoil,
        spacing_to_chord=1.0,
        leading_edge_gap_to_chord=0.25,
        trailing_edge_gap_to_chord=0.25,
        type="camber",
        mesh_params=TurboMeshParameters(
            airfoil_label="wall1",
            bottom_label="periodic1",
            top_label="periodic2",
            inlet_label="inflow",
            outlet_label="outmix",
        )
    ),
    outflow_passage=TurboRowPassage(
        airfoil=stator_airfoil,
        spacing_to_chord=1.0,
        leading_edge_gap_to_chord=0.25,
        trailing_edge_gap_to_chord=0.25,
        type="camber",
        mesh_params=TurboMeshParameters(
            airfoil_label="wall2",
            bottom_label="periodic3",
            top_label="periodic4",
            inlet_label="inmix",
            outlet_label="outflow",
        )
    )
)

# axial_turbo_passage.inflow_passage.visualize()
# axial_turbo_passage.outflow_passage.visualize()

from ezmesh.exporters import export_to_su2


flasher = get_flasher("air", "gas")
inflow = flasher.flash(P=87909, T=276, mach_number=0.4098)

meshes = axial_turbo_passage.get_meshes(SimulationOptions(inflow))
export_to_su2(meshes, "./simulation_out/passage1.su2")
visualize_mesh(meshes)


2.361371219311666e-06
2.361371219311666e-06
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Nurb)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Nurb)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Line)
Info    : [ 20%] Meshing curve 13 (Line)
Info    : [ 20%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Line)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Line)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 20%] Meshing curve 19 (Line)
Info    : [ 20%] Meshing curve 20 (Line)
Info    : [ 20%] Meshing curve 21 (Line)
Info    : [ 30%] Meshing curve 22 (Line)
Info    : [ 30%] Meshing curve 23 (Line

/usr/local/lib/python3.10/site-packages/pythreejs/traits.py:257: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


HTML(value='Coords: ()')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, far=100000.0, near=0.001, position=(0.0, 0.0, 1.0…

,outmix
,periodic2
,periodic1
,inflow
,wall1
,inmix
,periodic3
,periodic4
,outflow
,wall2
,Zone 0


In [2]:
# from parafoil.optimize import BaseOptimizer
# from paraflow import run_simulation, get_flasher, SimulationOptions, get_frames, display_frame

# flasher = get_flasher("air", "gas")

# optimizer = BaseOptimizer(
#     working_directory="./simulation_out",
#     passage=axial_turbo_passage,
#     sim_options=SimulationOptions(
#         inlet_total_state=flasher.flash(P=87909, T=276, mach_number=0.4098), 
#         target_outlet_static_state=flasher.flash(P=108932, T=296),
#         translation=[
#             np.array([0.0, 266, 0.0]),
#             None
#         ]
#     ),
#     objectives=[("efficiency", "max")]
# )

# optimizer.optimize()

In [3]:
from paraflow import run_simulation, get_flasher, SimulationOptions, get_frames, display_frame
from paraflow import SimulationResult

flasher = get_flasher("air", "gas")
sim_results = run_simulation(
    axial_turbo_passage,
    sim_options=SimulationOptions(
        inlet_total_state=flasher.flash(P=87909, T=276, mach_number=0.4098), 
        target_outlet_static_state=flasher.flash(P=108932, T=296),
        translation=[
            np.array([0.0, 266, 0.0]),
            None
        ]
    ),
    working_directory="./simulation_out", 
    id="1",
    auto_delete=False,
    simulator_config={
        "custom_executable_path": "./SU2_CFD"
    }
)
sim_results.to_file("./simulation_out/cached.pkl")

# sim_results = SimulationResult.from_file("./simulation_out/cached.pkl")

frames = get_frames(
    sim_results, 
    property_names=["Pressure"], 
    num_pnts=500,
)
display_frame(frames, "Pressure")
sim_results.eval_values.loc[sim_results.eval_values.index[-1], '  "TotTotEff[1]"  ']


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Nurb)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Nurb)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Line)
Info    : [ 20%] Meshing curve 13 (Line)
Info    : [ 20%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Line)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Line)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 20%] Meshing curve 19 (Line)
Info    : [ 20%] Meshing curve 20 (Line)
Info    : [ 20%] Meshing curve 21 (Line)
Info    : [ 30%] Meshing curve 22 (Line)
Info    : [ 30%] Meshing curve 23 (Line)
Info    : [ 30%] Meshing curve 24 (Line)
I

/usr/local/lib/python3.10/site-packages/pythreejs/traits.py:257: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


HTML(value='Coords: ()')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, far=100000.0, near=0.001, position=(0.0, 0.0, 1.0…

,periodic1
,inflow
,outmix
,periodic2
,wall1
,periodic4
,inmix
,outflow
,periodic3
,wall2
,Zone 0


Setting up SU2 Simulation for /workspace/parafoil/simulation_out/config1.cfg
Running SU2 Simulation for /workspace/parafoil/simulation_out/config1.cfg
